In [1]:
# Identify athletes meeting OCTC selection rules

#1. At least 1 athlete per event per gender
#2. Except for 100m, 400m, cap at 3 athletes per event. 100m, 400m capped at 6
#3. Where top athlete is >30 yrs old (except marathon), to include next athlete as well (below 30)
#4. Where althlete qualified in 2 events, to choose the better performing one
#5. For athletes looking to do full time, to write in to SAA for special consideration
#6. Exclude SPEX carded athletes
#7. Except for marathon, age threshold cut off of 40 yrs old for top athlete
#8. No double tapping of prog - potential names in red


%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime



In [3]:
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, AGE, RANK AS COMPETITION_RANK, EVENT, DOB, COUNTRY, CATEGORY_EVENT, GENDER, COMPETITION, DATE
FROM `saa-analytics.results.saa_full` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='FOUL' AND RANK!='DNS' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL
"""

athletes = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [100]:
athletes.to_csv('athletes.csv', sep=',', encoding='utf-8-sig', index=False)

In [4]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,"NG, YEW CHEO",47:32.31,37,4,Mixed 10000 Meter Run OPEN,1986-11-30,None,Long,X,SA Allcomers 4,2023
1,"SAVIOUR, GILBERT DOMINIC",39:43.41,18,1,Mixed 10000 Meter Run OPEN,2005-10-20,None,Long,X,SA Allcomers 4,2023
2,"Mohd Amin, Abdul Haq",44:50.30,23,3,Mixed 10000 Meter Run OPEN,2000-04-04,None,Long,X,SA Allcomers 4,2023
3,"Lee, Aaron",43:48.04,26,2,Mixed 10000 Meter Run OPEN,1997-12-31,None,Long,X,SA Allcomers 4,2023
4,"Shing Ling, Goh",39:28.78,25,8,Mixed 10000 Meter Run Open,1999-06-07,None,Long,X,SA Allcomers Meet 1,2024
...,...,...,...,...,...,...,...,...,...,...,...
25540,CEDAR GIRLS SECONDARY SCHOOL,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,NSG,2022
25541,HWA CHONG INSTITUTION,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,NSG,2022
25542,CEDAR GIRLS SECONDARY SCHOOL,04:26.2,None,1,4 X 400m relay,None,None,Relay,Female,NSG,2022
25543,SINGAPORE SPORTS SCHOOL,04:11.7,None,1,4 X 400m relay,None,None,Relay,Female,NSG,2022


In [11]:
# Run events

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Dash.+', value='200m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Dash.+', value='100m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Dash.+', value='400m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+800 Meter Run.+', value='800m')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Run.+', value='1500m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Run.+', value='3000m')

# Hurdles events

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110 Meter Hurdles.+', value='110m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110m Hurdles.+', value='110m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Hurdles.+', value='100m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Hurdles.+', value='200m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Hurdles.+', value='400m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400m Hurdles.+', value='400m hurdles')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x100 Meter Relay.+', value='4 x 100m relay')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x400 Meter Relay.+', value='4 x 400m relay')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 meter.+', value='3000m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Race Walk.+', value='1500m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000m Race Walk.+', value='3000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Race Walk.+', value='5000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000m Race Walk.+', value='5000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Race Walk.+', value='10000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Run.+', value='5000m run')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Run.+', value='10000m run')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Race Walk.+', value='race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Pole Vault.+', value='Pole vault')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Shot Put.+', value='Shot put')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Discus.+', value='Discus throw')

#mask = athletes['EVENT'].str.contains(r'Discus', na=True)
#athletes.loc[mask, 'EVENT'] = 'Discus throw'

#mask = athletes['EVENT'].str.contains(r'Shot', na=True)
#athletes.loc[mask, 'EVENT'] = 'Shot put'

#mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
#athletes.loc[mask, 'EVENT'] = 'Javelin throw'



#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Triple Jump.+', value='Triple jump')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw.+', value='Javelin throw')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump.+', value='Long jump')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Long Jump', value='Long jump')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'High Jump', value='High jump')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump.+', value='High jump')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+S/C.+', value='steeplechase')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Steeplechase.+', value='3000m steeplechase')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+2000 Meter Steeplechase.+', value='2000m steeplechase')


#mask = athletes['EVENT'].str.contains(r'High', na=True)
#athletes.loc[mask, 'EVENT'] = 'High jump'

#mask = athletes['EVENT'].str.contains(r'110m hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '110m hurdles'

#mask = athletes['EVENT'].str.contains(r'400m hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '400m hurdles'

#mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '200m hurdles'

#mask = athletes['EVENT'].str.contains(r'100m Hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '100m hurdles'

#mask = athletes['EVENT'].str.contains(r'4 X 100m relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

#mask = athletes['EVENT'].str.contains(r'4 X 400m relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 400m relay'

#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'EVENT'] = '2000m steeplechase'

#mask = athletes['EVENT'].str.contains(r'Hammer Throw', na=True)
#athletes.loc[mask, 'EVENT'] = 'Hammer throw'

#mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
#athletes.loc[mask, 'EVENT'] = '3000m steeplechase'

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'EVENT'] = '2000m steeplechase'


#mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

#mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 400m relay'


# correct javelin category
#mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
#athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'




In [5]:
# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

mask = athletes['EVENT'].str.contains(r'50 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
mask = athletes['EVENT'].str.contains(r'60 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
mask = athletes['EVENT'].str.contains(r'80 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = athletes['EVENT'].str.contains(r'100 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'200 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = athletes['EVENT'].str.contains(r'400 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'400m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'600 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = athletes['EVENT'].str.contains(r'800 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'1500 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'1500m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'3000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'3000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'5000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'5000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'10000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes['EVENT'].str.contains(r'10000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes['EVENT'].str.contains(r'1 Mile Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1 mile'


# Hurdles

mask = athletes['EVENT'].str.contains(r'80m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'80m hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'80 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'100m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'100 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'110m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
mask = athletes['EVENT'].str.contains(r'110 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'200 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'400m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'


# Throws

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Shot', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'Hammer', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer throw'
mask = athletes['EVENT'].str.contains(r'Discus', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'

# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High jump'
mask = athletes['EVENT'].str.contains(r'Long Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'
mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple jump'
mask = athletes['EVENT'].str.contains(r'Pole vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole vault'

# Steeplechase

mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m steeplechase'
mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m steeplechase'
mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m steeplechase'
mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m steeplechase'
mask = athletes['EVENT'].str.contains(r'3000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m steeplechase'


# Walk

mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m race walk'
mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m race walk'
mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m race walk'
mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m race walk'
mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m race walk'
mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m race walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4x400 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'


In [6]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,MAPPED_EVENT
0,"NG, YEW CHEO",47:32.31,37,4,Mixed 10000 Meter Run OPEN,1986-11-30,None,Long,X,SA Allcomers 4,2023,10000m
1,"SAVIOUR, GILBERT DOMINIC",39:43.41,18,1,Mixed 10000 Meter Run OPEN,2005-10-20,None,Long,X,SA Allcomers 4,2023,10000m
2,"Mohd Amin, Abdul Haq",44:50.30,23,3,Mixed 10000 Meter Run OPEN,2000-04-04,None,Long,X,SA Allcomers 4,2023,10000m
3,"Lee, Aaron",43:48.04,26,2,Mixed 10000 Meter Run OPEN,1997-12-31,None,Long,X,SA Allcomers 4,2023,10000m
4,"Shing Ling, Goh",39:28.78,25,8,Mixed 10000 Meter Run Open,1999-06-07,None,Long,X,SA Allcomers Meet 1,2024,10000m
...,...,...,...,...,...,...,...,...,...,...,...,...
25540,CEDAR GIRLS SECONDARY SCHOOL,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,NSG,2022,100m
25541,HWA CHONG INSTITUTION,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,NSG,2022,100m
25542,CEDAR GIRLS SECONDARY SCHOOL,04:26.2,None,1,4 X 400m relay,None,None,Relay,Female,NSG,2022,400m
25543,SINGAPORE SPORTS SCHOOL,04:11.7,None,1,4 X 400m relay,None,None,Relay,Female,NSG,2022,400m


In [98]:
athletes.to_csv('athletes2.csv', sep=',', encoding='utf-8-sig', index=False)


In [7]:
credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND RANK='3'
"""

benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [8]:
benchmarks

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
0,Muhammad Haiqal Hanafi,10.443,3,100m,Sprint,Male,SEA Games,Final
1,Lalu Muhammad Zohri,21.02,3,200m,Sprint,Male,SEA Games,Final
2,Frederick Ramirez,46.63,3,400m,Sprint,Male,SEA Games,Final
3,Wan Muhammad Fazri Wan Zahari,1:53.86,3,800m,Mid,Male,SEA Games,Final
4,Robi Syianturi,14:43.45,3,5000m,Long,Male,SEA Games,Final
5,Tan Htike Soe,31:25.55,3,10000m,Sprint,Male,SEA Games,Final
6,John Cabang,13.855,3,110m hurdles,Hurdles,Male,SEA Games,Final
7,Calvin Quek,50.75,3,400m hurdles,Hurdles,Male,SEA Games,Final
8,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final
9,"Jonathan Nyepa, Khairul Hafiz Jantan, Mohama...",39.36,3,4 x 100m relay,Relay,Male,SEA Games,Final


In [9]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [10]:
benchmarks.drop(['NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [11]:
benchmarks

,BENCHMARK,EVENT,GENDER
0,10.443,100m,Male
1,21.02,200m,Male
2,46.63,400m,Male
3,1:53.86,800m,Male
4,14:43.45,5000m,Male
5,31:25.55,10000m,Male
6,13.855,110m hurdles,Male
7,50.75,400m hurdles,Male
8,8:55.05,3000m steeplechase,Male
9,39.36,4 x 100m relay,Male


In [12]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()    
    
    if any(s in string for s in l)==True:
        
        if 'm' in metric:
            
            metric=metric.replace('m', '')
            output=float(str(metric))
            
        elif 'GR' in metric:
            
            metric=metric.replace('GR', '')
            output=float(str(metric))


        else:
    
            output=float(str(metric))
        
    else:
        
        searchstring = ":"
        searchstring2 = "."
        substring=str(metric)
        count = substring.count(searchstring)
        count2 = substring.count(searchstring2)
            
        if count==0:
            output=float(substring)
                        
             
        elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                                                
            time=str(metric)
            h, m ,s = time.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
        elif (count==1 and count2==1):
            
            m,s = metric.split(':')
            output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
            
            if output==229.90:
                print(metric, m, s, output, 'here')

                     
        elif (count==1 and count2==2):
            
            metric = metric.replace(".", ":", 1)
            
            h,m,s = metric.split(':')            
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
        elif (count==2 and count2==0):
            
            h,m,s = metric.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                
    return output

In [13]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,1]
    
    metric=benchmarks.iloc[rowIndex,0]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

In [14]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric
0,10.443,100m,Male,10.443
1,21.02,200m,Male,21.020
2,46.63,400m,Male,46.630
3,1:53.86,800m,Male,113.860
4,14:43.45,5000m,Male,883.450
5,31:25.55,10000m,Male,1885.550
6,13.855,110m hurdles,Male,13.855
7,50.75,400m hurdles,Male,50.750
8,8:55.05,3000m steeplechase,Male,535.050
9,39.36,4 x 100m relay,Male,39.360


In [15]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put', na=True)

benchmarks.loc[mask, '2pc']=benchmarks['Metric']*0.98
benchmarks.loc[mask, '35pc']=benchmarks['Metric']*0.965
benchmarks.loc[mask, '5pc']=benchmarks['Metric']*0.95

benchmarks.loc[~mask, '2pc']=benchmarks['Metric']*1.02
benchmarks.loc[~mask, '35pc']=benchmarks['Metric']*1.035
benchmarks.loc[~mask, '5pc']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [16]:
benchmarks['MAPPED_EVENT']=benchmarks['EVENT']

In [17]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2pc,35pc,5pc,MAPPED_EVENT
0,10.443,100m,Male,10.443,10.65186,10.808505,10.96515,100m
1,21.02,200m,Male,21.020,21.44040,21.755700,22.07100,200m
2,46.63,400m,Male,46.630,47.56260,48.262050,48.96150,400m
3,1:53.86,800m,Male,113.860,116.13720,117.845100,119.55300,800m
4,14:43.45,5000m,Male,883.450,901.11900,914.370750,927.62250,5000m
5,31:25.55,10000m,Male,1885.550,1923.26100,1951.544250,1979.82750,10000m
6,13.855,110m hurdles,Male,13.855,14.13210,14.339925,14.54775,110m hurdles
7,50.75,400m hurdles,Male,50.750,51.76500,52.526250,53.28750,400m hurdles
8,8:55.05,3000m steeplechase,Male,535.050,545.75100,553.776750,561.80250,3000m steeplechase
9,39.36,4 x 100m relay,Male,39.360,40.14720,40.737600,41.32800,4 x 100m relay


In [18]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,MAPPED_EVENT
0,"NG, YEW CHEO",47:32.31,37,4,Mixed 10000 Meter Run OPEN,1986-11-30,None,Long,X,SA Allcomers 4,2023,10000m
1,"SAVIOUR, GILBERT DOMINIC",39:43.41,18,1,Mixed 10000 Meter Run OPEN,2005-10-20,None,Long,X,SA Allcomers 4,2023,10000m
2,"Mohd Amin, Abdul Haq",44:50.30,23,3,Mixed 10000 Meter Run OPEN,2000-04-04,None,Long,X,SA Allcomers 4,2023,10000m
3,"Lee, Aaron",43:48.04,26,2,Mixed 10000 Meter Run OPEN,1997-12-31,None,Long,X,SA Allcomers 4,2023,10000m
4,"Shing Ling, Goh",39:28.78,25,8,Mixed 10000 Meter Run Open,1999-06-07,None,Long,X,SA Allcomers Meet 1,2024,10000m
...,...,...,...,...,...,...,...,...,...,...,...,...
25540,CEDAR GIRLS SECONDARY SCHOOL,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,NSG,2022,100m
25541,HWA CHONG INSTITUTION,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,NSG,2022,100m
25542,CEDAR GIRLS SECONDARY SCHOOL,04:26.2,None,1,4 X 400m relay,None,None,Relay,Female,NSG,2022,400m
25543,SINGAPORE SPORTS SCHOOL,04:11.7,None,1,4 X 400m relay,None,None,Relay,Female,NSG,2022,400m


In [19]:
df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [20]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,MAPPED_EVENT,index_y,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc
0,0,"NG, YEW CHEO",47:32.31,37,4,Mixed 10000 Meter Run OPEN,1986-11-30,None,Long,X,SA Allcomers 4,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"SAVIOUR, GILBERT DOMINIC",39:43.41,18,1,Mixed 10000 Meter Run OPEN,2005-10-20,None,Long,X,SA Allcomers 4,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"Mohd Amin, Abdul Haq",44:50.30,23,3,Mixed 10000 Meter Run OPEN,2000-04-04,None,Long,X,SA Allcomers 4,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Lee, Aaron",43:48.04,26,2,Mixed 10000 Meter Run OPEN,1997-12-31,None,Long,X,SA Allcomers 4,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"Shing Ling, Goh",39:28.78,25,8,Mixed 10000 Meter Run Open,1999-06-07,None,Long,X,SA Allcomers Meet 1,2024,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25540,25540,CEDAR GIRLS SECONDARY SCHOOL,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,NSG,2022,100m,21.0,11.75,100m,11.75,11.9850,12.16125,12.3375
25541,25541,HWA CHONG INSTITUTION,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,NSG,2022,100m,21.0,11.75,100m,11.75,11.9850,12.16125,12.3375
25542,25542,CEDAR GIRLS SECONDARY SCHOOL,04:26.2,None,1,4 X 400m relay,None,None,Relay,Female,NSG,2022,400m,23.0,53.84,400m,53.84,54.9168,55.72440,56.5320
25543,25543,SINGAPORE SPORTS SCHOOL,04:11.7,None,1,4 X 400m relay,None,None,Relay,Female,NSG,2022,400m,23.0,53.84,400m,53.84,54.9168,55.72440,56.5320


In [21]:
df[df['MAPPED_EVENT']=='High jump']

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,MAPPED_EVENT,index_y,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc
4078,4078,WANG LINGJUN,1.81,None,4.0,High Jump,None,None,Jump,Male,NSG,2023,High jump,13.0,2.15 m,High jump,2.15,2.1070,2.07475,2.0425
4079,4079,"FUN LE CONG, LUCAS",1.84,None,2.0,High Jump,None,None,Jump,Male,NSG,2023,High jump,13.0,2.15 m,High jump,2.15,2.1070,2.07475,2.0425
4080,4080,ASHTON TAN YIWEI,1.9,None,1.0,High Jump,None,None,Jump,Male,NSG,2023,High jump,13.0,2.15 m,High jump,2.15,2.1070,2.07475,2.0425
4081,4081,ASHER WONG,1.84,None,3.0,High Jump,None,None,Jump,Male,NSG,2023,High jump,13.0,2.15 m,High jump,2.15,2.1070,2.07475,2.0425
4082,4082,DILLON LIM YUN KAI,1.7,None,7.0,High Jump,None,None,Jump,Male,NSG,2023,High jump,13.0,2.15 m,High jump,2.15,2.1070,2.07475,2.0425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25468,25468,ASHER WONG,1.86,None,1,High jump,None,None,Jump,Male,NSG,2022,High jump,13.0,2.15 m,High jump,2.15,2.1070,2.07475,2.0425
25469,25469,TAN SHOU YI REI (CHEN SHOUYI),1.85,None,1,High jump,None,None,Jump,Male,NSG,2022,High jump,13.0,2.15 m,High jump,2.15,2.1070,2.07475,2.0425
25470,25470,ABEL LEE BING HENG,2,None,1,High jump,None,None,Jump,Male,NSG,2022,High jump,13.0,2.15 m,High jump,2.15,2.1070,2.07475,2.0425
25523,25523,CHONG LE XUAN CHLOE,1.56,None,1,High jump,None,None,Jump,Female,NSG,2022,High jump,35.0,1.73,High jump,1.73,1.6954,1.66945,1.6435


In [22]:
df['RESULT'] = athletes['RESULT'].replace(regex=r'–', value=np.NaN)


In [23]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,MAPPED_EVENT,index_y,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc
0,0,"NG, YEW CHEO",47:32.31,37,4,Mixed 10000 Meter Run OPEN,1986-11-30,None,Long,X,SA Allcomers 4,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"SAVIOUR, GILBERT DOMINIC",39:43.41,18,1,Mixed 10000 Meter Run OPEN,2005-10-20,None,Long,X,SA Allcomers 4,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"Mohd Amin, Abdul Haq",44:50.30,23,3,Mixed 10000 Meter Run OPEN,2000-04-04,None,Long,X,SA Allcomers 4,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Lee, Aaron",43:48.04,26,2,Mixed 10000 Meter Run OPEN,1997-12-31,None,Long,X,SA Allcomers 4,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"Shing Ling, Goh",39:28.78,25,8,Mixed 10000 Meter Run Open,1999-06-07,None,Long,X,SA Allcomers Meet 1,2024,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25540,25540,CEDAR GIRLS SECONDARY SCHOOL,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,NSG,2022,100m,21.0,11.75,100m,11.75,11.9850,12.16125,12.3375
25541,25541,HWA CHONG INSTITUTION,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,NSG,2022,100m,21.0,11.75,100m,11.75,11.9850,12.16125,12.3375
25542,25542,CEDAR GIRLS SECONDARY SCHOOL,04:26.2,None,1,4 X 400m relay,None,None,Relay,Female,NSG,2022,400m,23.0,53.84,400m,53.84,54.9168,55.72440,56.5320
25543,25543,SINGAPORE SPORTS SCHOOL,04:11.7,None,1,4 X 400m relay,None,None,Relay,Female,NSG,2022,400m,23.0,53.84,400m,53.84,54.9168,55.72440,56.5320


In [122]:
df.to_csv('df.csv', sep=',', encoding='utf-8-sig', index=False)


In [24]:
# Convert results into seconds format

for i in range(len(df)):
    
        
    rowIndex = df.index[i]

    input_string=df.iloc[rowIndex,5]    
    
    metric=df.iloc[rowIndex,2]
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric==' DNS' or metric=='NH':
        continue
    
        
    out = convert_time(i, input_string, metric)
         
    df.loc[rowIndex, 'RESULT_CONV'] = out

In [25]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,DATE,MAPPED_EVENT,index_y,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV
0,0,"NG, YEW CHEO",47:32.31,37,4,Mixed 10000 Meter Run OPEN,1986-11-30,None,Long,X,...,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2852.31
1,1,"SAVIOUR, GILBERT DOMINIC",39:43.41,18,1,Mixed 10000 Meter Run OPEN,2005-10-20,None,Long,X,...,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2383.41
2,2,"Mohd Amin, Abdul Haq",44:50.30,23,3,Mixed 10000 Meter Run OPEN,2000-04-04,None,Long,X,...,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2690.30
3,3,"Lee, Aaron",43:48.04,26,2,Mixed 10000 Meter Run OPEN,1997-12-31,None,Long,X,...,2023,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2628.04
4,4,"Shing Ling, Goh",39:28.78,25,8,Mixed 10000 Meter Run Open,1999-06-07,None,Long,X,...,2024,10000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2368.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25540,25540,CEDAR GIRLS SECONDARY SCHOOL,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,...,2022,100m,21.0,11.75,100m,11.75,11.9850,12.16125,12.3375,50.10
25541,25541,HWA CHONG INSTITUTION,00:50.1,None,1,4 X 100m relay,None,None,Relay,Female,...,2022,100m,21.0,11.75,100m,11.75,11.9850,12.16125,12.3375,50.10
25542,25542,CEDAR GIRLS SECONDARY SCHOOL,04:26.2,None,1,4 X 400m relay,None,None,Relay,Female,...,2022,400m,23.0,53.84,400m,53.84,54.9168,55.72440,56.5320,266.20
25543,25543,SINGAPORE SPORTS SCHOOL,04:11.7,None,1,4 X 400m relay,None,None,Relay,Female,...,2022,400m,23.0,53.84,400m,53.84,54.9168,55.72440,56.5320,251.70


In [26]:
df["AGE"].fillna(0, inplace=True)
df['AGE'] = df['AGE'].astype('float')

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45857/2753303150.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["AGE"].fillna(0, inplace=True)


In [1974]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [1975]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [27]:
rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Mid')|(df['CATEGORY_EVENT']=='Sprint')|(df['CATEGORY_EVENT']=='Long')|(df['CATEGORY_EVENT']=='Hurdles')|(df['CATEGORY_EVENT']=='Walk')|(df['CATEGORY_EVENT']=='Relay')|(df['CATEGORY_EVENT']=='Marathon')|(df['CATEGORY_EVENT']=='Steeple')|(df['CATEGORY_EVENT']=='Pentathlon')|(df['CATEGORY_EVENT']=='Heptathlon')|(df['CATEGORY_EVENT']=='Triathlon'))&(df['RESULT_CONV'] <= df['5pc']) & (df['AGE']<40) & ((df['MAPPED_EVENT']!='Marathon')|(df['AGE']<60) & (df['MAPPED_EVENT']=='Marathon')))|(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['MAPPED_EVENT']!='Marathon')|(df['AGE']<60) & (df['MAPPED_EVENT']=='Marathon')))]

In [28]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,DATE,MAPPED_EVENT,index_y,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV
319,319,"LOH DING RONG, ANSON",50.25,0.0,1.0,Discus (1.50kg),None,None,Throw,Male,...,2023,Discus throw,18.0,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,50.25
322,322,TAN JUN JIE,48.38,0.0,2.0,Discus (1.50kg),None,None,Throw,Male,...,2023,Discus throw,18.0,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,48.38
339,339,"CHUA JE-AN, GARRETT",00:14.3,0.0,1.0,110m Hurdles (0.914m),None,None,Hurdles,Male,...,2023,110m hurdles,6.0,13.855,110m hurdles,13.855,14.1321,14.339925,14.54775,14.30
423,423,LEE YU FOONG (LI YUFENG),51.28,0.0,1.0,Discus (1.00kg),None,None,Throw,Male,...,2023,Discus throw,18.0,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,51.28
512,512,Tan Jun Jie,53.4,0.0,1.0,Discus (1.50kg),None,None,Throw,Male,...,2024,Discus throw,18.0,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,53.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25447,25447,MARK LEE REN,00:21.6,0.0,1,200m,None,None,Sprint,Male,...,2022,200m,1.0,21.02,200m,21.020,21.4404,21.755700,22.07100,21.60
25460,25460,LOH DING RONG ANSON,58.88,0.0,1,Discus,None,None,Throw,Male,...,2022,Discus throw,18.0,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,58.88
25461,25461,LOH JING JIE ALOYSIUS,49.55,0.0,1,Discus,None,None,Throw,Male,...,2022,Discus throw,18.0,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,49.55
25467,25467,LOH JING JIE ALOYSIUS,17.89,0.0,1,Shot put,None,None,Throw,Male,...,2022,Shot put,17.0,17.3,Shot put,17.300,16.9540,16.694500,16.43500,17.89


In [29]:
mask = rslt_df['CATEGORY_EVENT'].str.contains(r'Jump|Throw', na=True)
rslt_df.loc[mask, 'Delta2'] = rslt_df['RESULT_CONV']-rslt_df['2pc']
rslt_df.loc[mask, 'Delta35'] = rslt_df['RESULT_CONV']-rslt_df['35pc']
rslt_df.loc[mask, 'Delta5'] = rslt_df['RESULT_CONV']-rslt_df['5pc']

rslt_df.loc[~mask, 'Delta2'] =  rslt_df['2pc'] - rslt_df['RESULT_CONV']
rslt_df.loc[~mask, 'Delta35'] = rslt_df['35pc'] - rslt_df['RESULT_CONV']
rslt_df.loc[~mask, 'Delta5'] = rslt_df['5pc'] - rslt_df['RESULT_CONV']



#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
rslt_df=rslt_df.loc[rslt_df['COMPETITION']!='SEA Games']

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45857/164990800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df.loc[mask, 'Delta2'] = rslt_df['RESULT_CONV']-rslt_df['2pc']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45857/164990800.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df.loc[mask, 'Delta35'] = rslt_df['RESULT_CONV']-rslt_df['35pc']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45857/164990800.py:4: SettingWithCopyWarning: 
A value is try

In [30]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5
319,319,"LOH DING RONG, ANSON",50.25,0.0,1.0,Discus (1.50kg),None,None,Throw,Male,...,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,50.25,1.2304,1.980700,2.73100
322,322,TAN JUN JIE,48.38,0.0,2.0,Discus (1.50kg),None,None,Throw,Male,...,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,48.38,-0.6396,0.110700,0.86100
339,339,"CHUA JE-AN, GARRETT",00:14.3,0.0,1.0,110m Hurdles (0.914m),None,None,Hurdles,Male,...,13.855,110m hurdles,13.855,14.1321,14.339925,14.54775,14.30,-0.1679,0.039925,0.24775
423,423,LEE YU FOONG (LI YUFENG),51.28,0.0,1.0,Discus (1.00kg),None,None,Throw,Male,...,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,51.28,2.2604,3.010700,3.76100
512,512,Tan Jun Jie,53.4,0.0,1.0,Discus (1.50kg),None,None,Throw,Male,...,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,53.40,4.3804,5.130700,5.88100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25447,25447,MARK LEE REN,00:21.6,0.0,1,200m,None,None,Sprint,Male,...,21.02,200m,21.020,21.4404,21.755700,22.07100,21.60,-0.1596,0.155700,0.47100
25460,25460,LOH DING RONG ANSON,58.88,0.0,1,Discus,None,None,Throw,Male,...,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,58.88,9.8604,10.610700,11.36100
25461,25461,LOH JING JIE ALOYSIUS,49.55,0.0,1,Discus,None,None,Throw,Male,...,50.02,Discus throw,50.020,49.0196,48.269300,47.51900,49.55,0.5304,1.280700,2.03100
25467,25467,LOH JING JIE ALOYSIUS,17.89,0.0,1,Shot put,None,None,Throw,Male,...,17.3,Shot put,17.300,16.9540,16.694500,16.43500,17.89,0.9360,1.195500,1.45500


In [31]:
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Ryan, Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'LOUIS, MARC BRIAN', value='Louis, Marc Brian')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Louis, Marc', value='Louis, Marc Brian')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'TAN JUN JIE', value='Tan Jun Jie')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'SNG, MICHELLE', value='Sng, Michelle')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'SNG, SUAT LI, MICHELLE', value='Sng, Michelle')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'MUN, IVAN', value='Mun, Ivan')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'LOW, JUN YU', value='Low, Jun Yu')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'ANG, CHEN XIANG', value='Ang, Chen Xiang')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'LIM, OLIVER', value='Lim, Oliver')



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45857/180801585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45857/180801585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Ryan, Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t400

In [32]:
rslt_df['PERF_SCALAR']=rslt_df['Delta5']/rslt_df['Metric']*100

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45857/1814660016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['PERF_SCALAR']=rslt_df['Delta5']/rslt_df['Metric']*100


In [33]:
rslt_df.to_csv('checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)


In [34]:
#rslt_df['Rank'] = (rslt_df.sort_values(by=['EVENT', 'GENDER', 'Delta35'], ascending=[False, False, True])['Delta35']
#                .rank(method='first', ascending=False)
#             )


In [35]:
spex_list=pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/SPEX_CARDED_LIST.csv', encoding='latin-1')


In [36]:
spex_list

,V1,V2,V3,V4,Level
0,Goh Chui Ling,"Goh, Chui Ling",NaN,NaN,E3
1,Michelle Sng,"Sng, Michelle",NaN,NaN,E3
2,Quek Jun Jie Calvin,"Quek, Jun Jie Calvin",NaN,NaN,E3
3,"SOH RUI YONG, GUILLAUME","Soh Rui Yong, Guillaume",NaN,NaN,E3
4,Aaron Justin tan wen jie,"Tan Wen Jie, Aaron Justin",NaN,NaN,E3P
5,DANIEL LEOW SOON YEE,"Yee, Daniel Leow Soon",NaN,NaN,E3P
6,Joshua Chua,"Chua, Joshua",NaN,NaN,E3P
7,NG ZHI RONG RYAN RAPHAEL,"Ng Zhi Rong, Ryan Raphael","Raphael, Ryan",NaN,E3P
8,Wenli Rachel,"Wenli, Rachel",NaN,NaN,E3P
9,Wong YaoHan Melvin,"Wong YaoHan, Melvin",NaN,NaN,E3P


In [37]:
spex1 = spex_list['V1'].dropna().tolist()
spex2 = spex_list['V2'].dropna().tolist()
spex3 = spex_list['V3'].dropna().tolist()
spex4 = spex_list['V4'].dropna().tolist()


In [38]:
spex_athletes=spex1+spex2+spex3+spex4

In [39]:
spex_athletes

['Goh Chui Ling',
 'Michelle Sng',
 'Quek Jun Jie Calvin',
 'SOH RUI YONG, GUILLAUME',
 'Aaron Justin tan wen jie',
 'DANIEL LEOW SOON YEE',
 'Joshua Chua',
 'NG ZHI RONG RYAN RAPHAEL',
 'Wenli Rachel',
 'Wong YaoHan Melvin',
 'XANDER HO ANN HENG',
 'Veronica Shanti Pereira',
 'Ang Chen Xiang',
 'KAMPTON KAM',
 'MARC BRIAN LOUIS',
 'Mark Lee Ren',
 'Reuben Rainer Lee Siong En',
 'ELIZABETH-ANN TAN SHEE RU',
 'Thiruben Thana Rajan',
 'Bhavna Gopikrishna',
 'Chloe Chee En-Ya',
 'Conrad Kangli Emery',
 'Harry Irfan Curran',
 'Huang weijun',
 'Jayden tan',
 'Koh Shun Yi Audrey',
 'LAAVINIA D/O JAIGANTH',
 'Lim Yee Chern Clara',
 'LOH DING RONG ANSON',
 'ONG YING TAT',
 'SONG EN XU REAGAN',
 'Subaraghav hari',
 'Teh Ying Shan',
 'YAN TEO',
 'ZHONG CHUHAN',
 'ESTHER TAY SHEE WEI',
 'Faith Ford',
 'Garrett Chua Je-An',
 'Lucas Fun',
 'Goh, Chui Ling',
 'Sng, Michelle',
 'Quek, Jun Jie Calvin',
 'Soh Rui Yong, Guillaume',
 'Tan Wen Jie, Aaron Justin',
 'Yee, Daniel Leow Soon',
 'Chua, Joshua',

In [40]:
spex_athletes_casefold=[s.casefold() for s in spex_athletes]

In [41]:
spex_athletes_casefold

['goh chui ling',
 'michelle sng',
 'quek jun jie calvin',
 'soh rui yong, guillaume',
 'aaron justin tan wen jie',
 'daniel leow soon yee',
 'joshua chua',
 'ng zhi rong ryan raphael',
 'wenli rachel',
 'wong yaohan melvin',
 'xander ho ann heng',
 'veronica shanti pereira',
 'ang chen xiang',
 'kampton kam',
 'marc brian louis',
 'mark lee ren',
 'reuben rainer lee siong en',
 'elizabeth-ann tan shee ru',
 'thiruben thana rajan',
 'bhavna gopikrishna',
 'chloe chee en-ya',
 'conrad kangli emery',
 'harry irfan curran',
 'huang weijun',
 'jayden tan',
 'koh shun yi audrey',
 'laavinia d/o jaiganth',
 'lim yee chern clara',
 'loh ding rong anson',
 'ong ying tat',
 'song en xu reagan',
 'subaraghav hari',
 'teh ying shan',
 'yan teo',
 'zhong chuhan',
 'esther tay shee wei',
 'faith ford',
 'garrett chua je-an',
 'lucas fun',
 'goh, chui ling',
 'sng, michelle',
 'quek, jun jie calvin',
 'soh rui yong, guillaume',
 'tan wen jie, aaron justin',
 'yee, daniel leow soon',
 'chua, joshua',

In [42]:
top_performers=rslt_df.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [43]:
top_performers

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR
20709,20709,"{80: 'Jirapat, KHANONTA\xa0', 79: 'Manatsada, ...",46.06,0.0,3,Women 4x100 Meter Relay Open,None,None,Relay,Female,...,4 x 100m relay,44.580,45.47160,46.140300,46.80900,46.06,-0.58840,0.080300,0.74900,1.680126
8425,8425,"{71: 'KANHAAUDOM, Methawi', 74: 'Joseph Obioma...",3:14.84,0.0,2,Men 4x400 Meter Relay Open,None,None,Relay,Male,...,4 x 400m relay,188.820,192.59640,195.428700,198.26100,194.84,-2.24360,0.588700,3.42100,1.811778
5331,5331,"{36: 'QUEK, CALVIN', 35: 'CHUA, JOSHUA', 37: '...",40.56,0.0,1,Men 4x100 Meter Relay Open,None,None,Relay,Male,...,4 x 100m relay,39.360,40.14720,40.737600,41.32800,40.56,-0.41280,0.177600,0.76800,1.951220
8415,8415,"{26: 'ROSLAN, ABDUL WAFIY', 29: 'ROSELI, RUSLE...",3:14.46,0.0,1,Men 4x400 Meter Relay Open,None,None,Relay,Male,...,4 x 400m relay,188.820,192.59640,195.428700,198.26100,194.46,-1.86360,0.968700,3.80100,2.013028
20715,20715,"{260: 'ZULKIFLI, ZAIDATUL HUSNIAH', 261: 'ALIA...",45.70,0.0,1,Women 4x100 Meter Relay Open,None,None,Relay,Female,...,4 x 100m relay,44.580,45.47160,46.140300,46.80900,45.70,-0.22840,0.440300,1.10900,2.487663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8258,8258,"ANURA, ANDRE",14.93m,24.0,2,Men Triple Jump Open,1999-06-12,None,Jump,Male,...,Triple jump,15.700,15.38600,15.150500,14.91500,14.93,-0.45600,-0.220500,0.01500,0.095541
8531,8531,"ANG, JAMIE EL-REDHA",10.80,20.0,5,Men 100 Meter Dash Open,2004-01-06 00:00:00.000,None,Sprint,Male,...,100m,10.443,10.65186,10.808505,10.96515,10.80,-0.14814,0.008505,0.16515,1.581442
20683,20683,"ALING, NUR AISHAH ROFINA",12.09,24.0,4,Women 100 Meter Dash Open,1999-01-31,None,Sprint,Female,...,100m,11.750,11.98500,12.161250,12.33750,12.09,-0.10500,0.071250,0.24750,2.106383
25471,25471,ABEL LEE BING HENG,7.26,0.0,1,Long jump,None,None,Jump,Male,...,Long jump,7.620,7.46760,7.353300,7.23900,7.26,-0.20760,-0.093300,0.02100,0.275591


In [44]:
top_performers.to_csv('top_checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)


In [45]:
spexed_list = top_performers.loc[~rslt_df['NAME'].str.casefold().isin(spex_athletes_casefold)]  # ~ means NOT IN. DROP spex carded athletes

In [46]:
#spexed_list=top_performers

In [47]:
spexed_list.sort_values(['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
spexed_list['overall_rank'] = 1
spexed_list['overall_rank'] = spexed_list.groupby(['MAPPED_EVENT', 'GENDER'])['overall_rank'].cumsum()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45857/1338601511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spexed_list.sort_values(['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45857/1338601511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spexed_list['overall_rank'] = 1
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45857/1338601511.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [48]:
#spexed_list=spexed_list[(((spexed_list['EVENT']=='400m')&(spexed_list['overall_rank']<7)))|(((spexed_list['EVENT']=='100m')&(spexed_list['overall_rank']<7)))]

In [49]:
#Apply OCTC selection rule: max 6 for 100m/400m and max 3 for all other events

spexed_list=spexed_list[(((spexed_list['MAPPED_EVENT']=='400m')|(spexed_list['MAPPED_EVENT']=='100m'))&(spexed_list['overall_rank']<7))|(~((spexed_list['MAPPED_EVENT']=='400m')|(spexed_list['MAPPED_EVENT']=='100m'))&(spexed_list['overall_rank']<4))]


In [50]:
spexed_list

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR,overall_rank
8674,8674,"Lim, Jon",32:54.40,0.0,1,Men 10000 Meter Run Open,None,None,Long,Male,...,1885.55,1923.261,1951.54425,1979.8275,1974.40,-51.139,-22.85575,5.4275,0.287847,1
20673,20673,"HU, Chia-Chen",11.57,26.0,2,Women 100 Meter Dash Open,1997-03-28,None,Sprint,Female,...,11.75,11.985,12.16125,12.3375,11.57,0.415,0.59125,0.7675,6.531915,1
20683,20683,"ALING, NUR AISHAH ROFINA",12.09,24.0,4,Women 100 Meter Dash Open,1999-01-31,None,Sprint,Female,...,11.75,11.985,12.16125,12.3375,12.09,-0.105,0.07125,0.2475,2.106383,2
20539,20539,"Sagi, Mayako",12.16,21.0,1,Women 100 Meter Dash Open,2003-01-03,None,Sprint,Female,...,11.75,11.985,12.16125,12.3375,12.16,-0.175,0.00125,0.1775,1.510638,3
20680,20680,"Sukanda, PETRAKSA",12.20,20.0,5,Women 100 Meter Dash Open,2003-08-15,None,Sprint,Female,...,11.75,11.985,12.16125,12.3375,12.20,-0.215,-0.03875,0.1375,1.170213,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25362,25362,CHOO XING KAI JOSHUA,16.88,0.0,2,Shot put,None,None,Throw,Male,...,17.30,16.954,16.69450,16.4350,16.88,-0.074,0.18550,0.4450,2.572254,2
8217,8217,"CHEN, Yi Chu",16.87m,21.0,1,Men Shot Put Open,2002-10-31,None,Throw,Male,...,17.30,16.954,16.69450,16.4350,16.87,-0.084,0.17550,0.4350,2.514451,3
8463,8463,"LEE, MINJE",15.20m,23.0,1,Men Triple Jump Open,2001-04-11 00:00:00.000,None,Jump,Male,...,15.70,15.386,15.15050,14.9150,15.20,-0.186,0.04950,0.2850,1.815287,1
8259,8259,"Ting Li King, Brandon",15.03m,21.0,1,Men Triple Jump Open,2002-01-09,None,Jump,Male,...,15.70,15.386,15.15050,14.9150,15.03,-0.356,-0.12050,0.1150,0.732484,2


In [52]:
spexed_list.to_csv('octc_all_events.csv', sep=',', encoding='utf-8-sig', index=False)


In [ ]:
# 1. If top athlete>30 (and the only one >30), there are already 6 in list, and next one is already <30 then it means do nothing? 
# 2. If top athlete>30 and is the sole pick.  Next athlete is far beyond 5% band.  Do we add?
# 3. If top athlete>30 and there are less than 6 within 5% band of SEAG benchmark. Do we add that one more who is beyond 5%?
# 4. At least one per gender.  The only pick is far beyond 5%.  Do we add?